In [1]:
import numpy as np

In [2]:
class Tensor(object):
    """docstring for tensor"""
    def __init__(self, initial_value,op,graph):
        self.initial_value = initial_value
        self.graph = graph
        self.op = op

    def __add__(self, other): # change the + operateor's methods, other is another self 
        return self.graph.add(self, other)

In [3]:
class BaseOp(object):
    """docstring for BaseOp"""
    def __init__(self, inputs,graph):
        self.inputs = [graph.convert(input_) for input_ in inputs]
        self.output = graph.tensor(op=self)
        self.graph  = graph

    def compute(self,sess,*args):
        raise NotImplementedError()
    def gradient(self,grad):
        raise NotImplementedError()
'''
class myParent( object ):
    def __init__( self, customParam ):
        self.parentNumber = 5
        self.customParam = customParam

class Child( myParent ):
    def __init__( self, customParam ):
        myParent.__init__( self, customParam )
        self.childNumber = 4        
'''        
        
class AddOp(BaseOp):

    
    def __init__(self, inputs,graph):
        BaseOp.__init__(self, inputs,graph)
        self.output = graph.convert(   self.compute(inputs[0],inputs[1])   ) #build a tensor object but no op
        self.output_tensor_with_op = graph.tensor(   self.compute(inputs[0],inputs[1]),op =self) # build tensor with op=self
        '''
        print("11111111111 is :"+str(inputs[0].initial_value))
        if inputs[0] is not None and inputs[1] is not None:
            self.output = graph.convert(   self.compute(inputs[0],inputs[1])   )
        #self.output = graph.convert(1000000000000000000000000000000)
    '''
    def compute(self,a=0,b=0):
        #a = self.inputs[0].initial_value
        #b = self.inputs[1].initial_value
        return a.initial_value + b.initial_value
    def gradient(self,grad):
        return [grad,grad]


In [29]:
class Graph(object):
	"""docstring for Graph"""
	def tensor(self,initial_value=None,op=None):
		return Tensor(initial_value=initial_value,op=op,graph=self)

	def convert(self,value):
		if isinstance(value,Tensor):
			return value
		return self.tensor(initial_value=value)

	def gradients(self,y,x_s):

		'''
		>>>queue.append(([1,1],1))
		>>>queue.pop(0)
		([1, 1], 1)



    	loss_op = graph.mean(graph.square(graph.transpose(y) - activations1))
    	parameters = [weights0, biases0, weights1, biases1]

    	gradients = graph.gradients(loss_op, parameters)
		'''
		queue = []
		queue.append((y,1)) #queue = [(y,1)]

		grads = {}

		while len(queue)>0:   #len(queue) = 1
			y,grad_y = queue.pop(0)  # y=y,grad_y =1
			grad_y = self.convert(grad_y) #convert grad_y = 1 to a tensor = 1
			#print(y.op)

			gradients = y.op.gradient(grad_y) #gradients = [grad_y = 1,grad_y = 1] grad_y is a tensor
			assert len(gradients) == len(y.op.inputs)#len(gradients) = 2, len(y.op.inputs) = 2, 
            #c = a + b + a + a + b/ y = c /  y.op.inputs is [a + b + a + a ,b]

			for tensor, gradient in zip(y.op.inputs, gradients):
                #zip( [a + b + a + a ,b] , [grad_y = 1,grad_y = 1]  )
				if tensor in grads:  # 1: tensor = a + b + a + a , grads = {} 
                    #2:tensor = b, grads={a + b + a + a:grad_y}
					grads[tensor] += gradient
				else:
					grads[tensor] = gradient # 1 : grads[a + b + a + a] = gradient(=grad_y) 
                    #2:grads[b] = gradient(=grad_y)

				if tensor.op:
					queue.append((tensor, gradient)) #queue=[(a + b + a + a,grad_y),(b,grad_y)]
                    #**********************************************************
                    # for b does not have a op, so tensor.op is None, so queue=[(a+b+a+a,grad_y)] 
                    # not have (b,grad_y)
                    #**********************************************************
                    
                    
            #while len(queue) = 2
            #queue.pop(0) = (a + b + a + a,grad_y) => y = a + b + a + a ,grad_y=grad_y=1(tensor)
            #......
            #while len(queue) = 3
            #queue.pop(0) = (b,grad_y) => y = b, grad_y = grad_y = 1
            

		return [grads[x] for x in x_s]


	def add(self,a,b):
		op = AddOp([a,b],graph=self)
		return op.output_tensor_with_op

In [17]:
class Session(object):
    def __init__(self, graph):
        self.graph = graph
        self.state = {}
        
    def run_op(self, op, context):
        args = [self.eval_tensor(tensor, context) for tensor in op.inputs]
        return op.compute(self, *args)
    
    def eval_tensor(self, tensor, context):
        
        if tensor not in context:
            if tensor.op is not None:
                context[tensor] = self.run_op(tensor.op, context) #let context add the value in op.input
            elif tensor in self.state and self.state[tensor] is not None:
                context[tensor] = self.state[tensor]              #let context add the value in self.state
            elif tensor not in self.state and tensor.initial_value is not None:
                context[tensor] = self.state[tensor] = tensor.initial_value
                                                        #let context add the value by the initial_value of tensor
        return context[tensor]
    def run(self, tensors, feed_dict=None):
        
        context = {}

        if feed_dict:
            context.update(feed_dict)

        return [self.eval_tensor(tensor, context) for tensor in tensors]

In [40]:
graph = Graph()
a = graph.tensor(2)
b = graph.tensor(5)
c = a + b + a + b
print(c.initial_value)
print("==================")
print(a.op)
print(b.op)
print(c.op)
print("==================")
print("Length is :"+str(len(c.op.inputs)))
print(c.op.inputs[0].op)
print(c.op.inputs[1].op)
print(c.op.inputs[0].initial_value)
print(c.op.inputs[1].initial_value)
print(c.op.output.initial_value)
print(c.op.output_with_op.initial_value)
print("==================")
print(c.initial_value)
print("==================")
grad_a, grad_b = graph.gradients(c, [a, b])
print(grad_a.initial_value)
print(grad_b.initial_value)
print("==================")
print("c.op.inputs[0].op.inputs[0] = a + b + a")
print(c.op.inputs[0].op.inputs[0].initial_value)
print(c.op.inputs[0].op.inputs[1].initial_value)
#sess = Session(graph)
#grad_a_, grad_b_ = sess.run([grad_a, grad_b], feed_dict={a: 2, b: 1})
#print(grad_a_)
#print(grad_b_)

14
None
None
Length is :2
None
9
5
14
14
14
2
2
c.op.inputs[0].op.inputs[0] = a + b + a
7
2


In [35]:
import numpy as np

class Tensor(object):
    """
    `Tensor` represents a _value_ in the graph. It's just a data container with
    methods for operator overloading (each of which delegate to the graph). It
    includes:

      - The initial value of the tensor.
      - The operation which produced the tensor, if applicable.
      - A reference to the graph this tensor belongs to.
    """

    def __init__(self, initial_value, op, graph):
        self.initial_value = initial_value
        self.graph = graph
        self.op = op

    # ## [Operator Overloading](https://docs.python.org/2/reference/datamodel.html?highlight=__radd__#emulating-numeric-types)
    def __add__(self, other):
        return self.graph.add(self, other)

class BaseOp(object):
    """
    `BaseOp` represents an operation that performs computation on tensors.
    Every operation consists of the following:

      - A list of `inputs`, each converted to ensure they're all tensors.
      - An output tensor to represent the result of the operation (which might
        be `None`.)
      - A reference to the graph so that each operation can generate new
        operations when constructing gradients.
    """

    def __init__(self, inputs, graph):
        self.inputs = [graph.convert(input_) for input_ in inputs]
        self.output = graph.tensor(op=self)
        self.graph = graph

    def compute(self, sess, *args):
        """
        The `compute` method receives as input the _evaluated_ input tensors
        and returns the result of performing its operation on the inputs.
        """
        raise NotImplementedError()

    def gradient(self, grad):
        """
        The `gradient` method computes the partial derivative w.r.t. each input
        to the operation. (Most of the derivatives come from
        [Wikipedia](https://en.wikipedia.org/wiki/Differentiation_rules).)
        """
        raise NotImplementedError()

class AddOp(BaseOp):
    """
    `AddOp` adds a tensor to another tensor. Uses the
    [sum rule](https://en.wikipedia.org/wiki/Sum_rule_in_differentiation) to
    compute the partial derivatives.
    """

    def compute(self, sess, a, b):
        return a + b

    def gradient(self, grad):
        return [grad, grad]
    
class Graph(object):
    """
    `Graph` represents a computation to be evaluated by a `Session`. With the
    exception of `Graph#tensor`, `Graph#convert`, and `Graph#gradients`, most
    methods simply create an operation and return the output tensor of the
    operation.
    """

    def tensor(self, initial_value=None, op=None):
        """
        The `tensor` method defines a new tensor with the given initial value
        and operation.
        """
        return Tensor(initial_value=initial_value, graph=self, op=op)

    def convert(self, value):
        """
        The `convert` method returns the given value if it is a `Tensor`,
        otherwise convert it to one.
        """
        if isinstance(value, Tensor):
            return value
        return self.tensor(initial_value=value)

    def add(self, a, b):
        op = AddOp([a, b], graph=self)
        return op.output
